# Crossmatch with ZTF and PanSTARRS

Crossmatch the Rubin data with ZTF and PanSTARRS and store the results as HATS catalogs on disk.

In [1]:
import os
import lsdb
import tempfile

from dask.distributed import Client
from pathlib import Path
from upath import UPath

In [2]:
VERSION = os.environ["VERSION"]
OUTPUT_DIR = Path(os.environ["OUTPUT_DIR"])

print(f"VERSION: {VERSION}")
print(f"OUTPUT_DIR: {OUTPUT_DIR}")

hats_dir = OUTPUT_DIR / "hats" / VERSION

In [3]:
tmp_path = tempfile.TemporaryDirectory()
tmp_dir = tmp_path.name
client = Client(n_workers=16, threads_per_worker=1, local_directory=tmp_dir)

Let"s load the nested Rubin catalogs:

In [4]:
dia_object_lc = lsdb.read_hats(hats_dir / "dia_object_collection")
dia_object_lc

In [5]:
object_lc = lsdb.read_hats(hats_dir / "object_collection")
object_lc

### Crossmatch with ZTF

In [6]:
# TODO

### Crossmatch with PanSTARRS

In [7]:
panstarrs = lsdb.read_hats(
    UPath("s3://stpubdata/panstarrs/ps1/public/hats/otmo", anon=True),
    margin_cache=UPath(
        "s3://stpubdata/panstarrs/ps1/public/hats/otmo_10arcs", anon=True
    ),
)
panstarrs

In [8]:
ps1_xmatch_radius_arcsec = 0.1

In [9]:
for catalog in [object_lc, dia_object_lc]:
    dist_column = "lsst_ztf_sep"
    catalog_name = f"{catalog.name}_x_ps1"
    lsst_lc_x_ps1_dr22 = catalog.crossmatch(
        panstarrs, radius_arcsec=ps1_xmatch_radius_arcsec, suffixes=("", "_ps1")
    )
    lsst_lc_x_ps1_dr22._ddf = lsst_lc_x_ps1_dr22._ddf.rename(
        columns={"_dist_arcsec": dist_column}
    )
    lsst_lc_x_ps1_dr22.to_hats(hats_dir / catalog_name, catalog_name=catalog_name)
    print(f"Saved {catalog_name}")

In [10]:
client.close()
tmp_path.cleanup()